In [1]:
%%writefile task1.cu
#include <iostream>
#include <cstdlib>
#include <ctime>

using namespace std;

// Размер массива
const int N = 1'000'000;

int main() {
    // Инициализация генератора случайных чисел
    srand(static_cast<unsigned int>(time(nullptr)));

    // Выделение памяти под массив
    int* data = new int[N];

    // Генерация случайных чисел
    for (int i = 0; i < N; i++) {
        data[i] = rand() % 100;
    }

    // Контрольный вывод первых элементов
    cout << "Первые 10 элементов массива:\n";
    for (int i = 0; i < 10; i++) {
        cout << data[i] << " ";
    }
    cout << endl;

    // Освобождение памяти
    delete[] data;

    return 0;
}


Writing task1.cu


In [2]:
!nvcc task1.cu -o task1
!./task1

Первые 10 элементов массива:
77 31 80 42 54 16 3 3 57 25 


In [3]:
%%writefile task2.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// Размер массива
const int N = 1'000'000;

// Размер блока потоков
const int BLOCK_SIZE = 256;

//  Вариант A
// Редукция с использованием только глобальной памяти.
// Каждый поток читает данные напрямую из global memory.
__global__ void reductionGlobal(const float* input, float* partial, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < n) {
        partial[idx] = input[idx];
    }
}

// - Вариант B
// Редукция с использованием разделяемой памяти.
// Данные блока сначала копируются в shared memory,
// затем выполняется суммирование внутри блока.
__global__ void reductionShared(const float* input, float* partial, int n) {
    __shared__ float sharedData[BLOCK_SIZE];

    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    int localIdx = threadIdx.x;

    if (globalIdx < n) {
        sharedData[localIdx] = input[globalIdx];
    } else {
        sharedData[localIdx] = 0.0f;
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (localIdx < stride) {
            sharedData[localIdx] += sharedData[localIdx + stride];
        }
        __syncthreads();
    }

    if (localIdx == 0) {
        partial[blockIdx.x] = sharedData[0];
    }
}

int main() {
    // Выделение памяти на CPU
    float* h_data = new float[N];

    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    // Выделение памяти на GPU
    float *d_data, *d_partial;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_partial, N * sizeof(float));

    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Глобальная память

    auto startGlobal = high_resolution_clock::now();
    reductionGlobal<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);
    cudaDeviceSynchronize();
    auto endGlobal = high_resolution_clock::now();

    double timeGlobal =
        duration<double, milli>(endGlobal - startGlobal).count();

    // Суммирование частичных результатов на CPU
    float* h_partial = new float[N];
    cudaMemcpy(h_partial, d_partial, N * sizeof(float), cudaMemcpyDeviceToHost);

    float sumGlobal = 0.0f;
    for (int i = 0; i < N; i++) {
        sumGlobal += h_partial[i];
    }

    // Разделяемая память

    auto startShared = high_resolution_clock::now();
    reductionShared<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);
    cudaDeviceSynchronize();
    auto endShared = high_resolution_clock::now();

    double timeShared =
        duration<double, milli>(endShared - startShared).count();

    cudaMemcpy(h_partial, d_partial, gridSize * sizeof(float),
               cudaMemcpyDeviceToHost);

    float sumShared = 0.0f;
    for (int i = 0; i < gridSize; i++) {
        sumShared += h_partial[i];
    }

    // Вывод результатов

    cout << "Размер массива: " << N << " элементов\n";
    cout << "Сумма (global memory): " << sumGlobal
         << " | Время: " << timeGlobal << " мс\n";
    cout << "Сумма (shared memory): " << sumShared
         << " | Время: " << timeShared << " мс\n";

    // Освобождение памяти
    cudaFree(d_data);
    cudaFree(d_partial);
    delete[] h_data;
    delete[] h_partial;

    return 0;
}


Writing task2.cu


In [4]:
!nvcc task2.cu -o task2
!./task2

Размер массива: 1000000 элементов
Сумма (global memory): 0 | Время: 49.1332 мс
Сумма (shared memory): 0 | Время: 0.018966 мс


In [9]:
%%writefile task3.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

// Размер массива
const int N = 1024;

// Размер подмассива для пузырьковой сортировки
const int SUBARRAY_SIZE = 32;

// Этап 1
// Пузырьковая сортировка небольших подмассивов.
// Используется локальная память потока.
__global__ void bubbleSortSubarrays(int* data) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    int start = idx * SUBARRAY_SIZE;

    if (start + SUBARRAY_SIZE <= N) {
        // Локальный массив в памяти потока
        int local[SUBARRAY_SIZE];

        // Загрузка данных из глобальной памяти
        for (int i = 0; i < SUBARRAY_SIZE; i++) {
            local[i] = data[start + i];
        }

        // Пузырьковая сортировка
        for (int i = 0; i < SUBARRAY_SIZE - 1; i++) {
            for (int j = 0; j < SUBARRAY_SIZE - i - 1; j++) {
                if (local[j] > local[j + 1]) {
                    int temp = local[j];
                    local[j] = local[j + 1];
                    local[j + 1] = temp;
                }
            }
        }

        // Запись результата обратно в глобальную память
        for (int i = 0; i < SUBARRAY_SIZE; i++) {
            data[start + i] = local[i];
        }
    }
}

// Этап 2
// Слияние двух соседних отсортированных подмассивов.
// Используется разделяемая память.
__global__ void mergeSubarrays(int* data, int* output) {
    __shared__ int shared[2 * SUBARRAY_SIZE];

    int blockStart = blockIdx.x * 2 * SUBARRAY_SIZE;

    // Загрузка данных в shared memory
    for (int i = threadIdx.x; i < 2 * SUBARRAY_SIZE; i += blockDim.x) {
        shared[i] = data[blockStart + i];
    }

    __syncthreads();

    // Слияние выполняется одним потоком блока
    if (threadIdx.x == 0) {
        int i = 0;
        int j = SUBARRAY_SIZE;
        int k = blockStart;

        while (i < SUBARRAY_SIZE && j < 2 * SUBARRAY_SIZE) {
            if (shared[i] < shared[j]) {
                output[k++] = shared[i++];
            } else {
                output[k++] = shared[j++];
            }
        }

        while (i < SUBARRAY_SIZE) {
            output[k++] = shared[i++];
        }

        while (j < 2 * SUBARRAY_SIZE) {
            output[k++] = shared[j++];
        }
    }
}

int main() {
    int* h_data = new int[N];

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        h_data[i] = rand() % 1000;
    }

    int* d_data;
    int* d_output;
    cudaMalloc(&d_data, N * sizeof(int));
    cudaMalloc(&d_output, N * sizeof(int));

    cudaMemcpy(d_data, h_data, N * sizeof(int), cudaMemcpyHostToDevice);

    // Запуск пузырьковой сортировки подмассивов
    int numSubarrays = N / SUBARRAY_SIZE;
    bubbleSortSubarrays<<<numSubarrays, 1>>>(d_data);

    // Запуск слияния подмассивов
    int numMerges = N / (2 * SUBARRAY_SIZE);
    mergeSubarrays<<<numMerges, SUBARRAY_SIZE>>>(d_data, d_output);

    cudaMemcpy(h_data, d_output, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Контрольный вывод первых элементов
    cout << "Первые 10 элементов отсортированного массива:\n";
    for (int i = 0; i < 10; i++) {
        cout << h_data[i] << " ";
    }
    cout << endl;

    // Освобождение памяти
    cudaFree(d_data);
    cudaFree(d_output);
    delete[] h_data;

    return 0;
}


Overwriting task3.cu


In [10]:
!nvcc task3.cu -o task3
!./task3

Первые 10 элементов отсортированного массива:
0 0 0 0 0 0 0 0 0 0 


In [12]:
%%writefile task4.cu
#include <iostream>
#include <cuda_runtime.h>
#include <chrono>

using namespace std;
using namespace chrono;

// Размер блока
const int BLOCK_SIZE = 256;

// Редукция с использованием только глобальной памяти
__global__ void reductionGlobal(const float* input, float* partial, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < n) {
        partial[idx] = input[idx];
    }
}

// Редукция с использованием разделяемой памяти
__global__ void reductionShared(const float* input, float* partial, int n) {
    __shared__ float sharedData[BLOCK_SIZE];

    int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
    int localIdx = threadIdx.x;

    if (globalIdx < n) {
        sharedData[localIdx] = input[globalIdx];
    } else {
        sharedData[localIdx] = 0.0f;
    }

    __syncthreads();

    for (int stride = blockDim.x / 2; stride > 0; stride >>= 1) {
        if (localIdx < stride) {
            sharedData[localIdx] += sharedData[localIdx + stride];
        }
        __syncthreads();
    }

    if (localIdx == 0) {
        partial[blockIdx.x] = sharedData[0];
    }
}

void runTest(int N) {
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    float *d_data, *d_partial;
    cudaMalloc(&d_data, N * sizeof(float));
    cudaMalloc(&d_partial, N * sizeof(float));
    cudaMemcpy(d_data, h_data, N * sizeof(float), cudaMemcpyHostToDevice);

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    auto startGlobal = high_resolution_clock::now();
    reductionGlobal<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);
    cudaDeviceSynchronize();
    auto endGlobal = high_resolution_clock::now();

    double timeGlobal =
        duration<double, milli>(endGlobal - startGlobal).count();

    auto startShared = high_resolution_clock::now();
    reductionShared<<<gridSize, BLOCK_SIZE>>>(d_data, d_partial, N);
    cudaDeviceSynchronize();
    auto endShared = high_resolution_clock::now();

    double timeShared =
        duration<double, milli>(endShared - startShared).count();

    cout << "Размер массива: " << N << endl;
    cout << "Global memory: " << timeGlobal << " мс" << endl;
    cout << "Shared memory: " << timeShared << " мс" << endl;
    cout << endl;

    cudaFree(d_data);
    cudaFree(d_partial);
    delete[] h_data;
}

int main() {
    runTest(10'000);
    runTest(100'000);
    runTest(1'000'000);
    return 0;
}


Writing task4.cu


In [13]:
!nvcc task4.cu -o task4
!./task4

Размер массива: 10000
Global memory: 7.51468 мс
Shared memory: 0.00222 мс

Размер массива: 100000
Global memory: 0.038103 мс
Shared memory: 0.001284 мс

Размер массива: 1000000
Global memory: 0.073338 мс
Shared memory: 0.002005 мс

